In [1]:
import numpy as np
import pandas as pd

In [8]:
data = pd.read_csv('data_prepared.csv', index_col=0)
test = pd.read_csv('test_prepaired.csv', index_col=0)

In [11]:
display(data.columns, test.columns)

Index(['id', 'actual_price', 'price', 'start_date', 'close_date',
       'sale_end_date', 'brand', 'model', 'generation', 'modification',
       'equipment', 'body_type', 'color', 'year', 'mileage', 'owners_count',
       'latitude', 'longitude', 'crashes', 'is_taxi', 'is_carsharing',
       'description', 'month', 'day', 'sale_year', 'day_of_week',
       'day_of_year', 'dist_moscow', 'sell_time'],
      dtype='object')

Index(['id', 'sale_end_date', 'brand', 'model', 'generation', 'modification',
       'equipment', 'body_type', 'color', 'year', 'mileage', 'owners_count',
       'latitude', 'longitude', 'crashes', 'is_taxi', 'is_carsharing',
       'description', 'month', 'day', 'sale_year', 'day_of_week',
       'day_of_year', 'dist_moscow', 'sell_time'],
      dtype='object')

In [12]:
X_train, y_train = data.drop(['id', 'price', 'actual_price', 'start_date', 'close_date', 'sale_end_date', 'latitude', 'longitude', 'sell_time'], axis=1), data['actual_price']

In [30]:
X_test = test.drop(['id', 'latitude', 'longitude', 'sell_time', 'sale_end_date'], axis=1)

In [31]:
# replace \n with space in description
X_test['description'] = X_test['description'].str.replace('\n', ' ')
# delete multiple spaces
X_test['description'] = X_test['description'].str.replace(' +', ' ')

C:\Users\aleks\AppData\Local\Temp\ipykernel_21052\1764344362.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  X_test['description'] = X_test['description'].str.replace(' +', ' ')


In [16]:
X_train.columns

Index(['brand', 'model', 'generation', 'modification', 'equipment',
       'body_type', 'color', 'year', 'mileage', 'owners_count', 'crashes',
       'is_taxi', 'is_carsharing', 'description', 'month', 'day', 'sale_year',
       'day_of_week', 'day_of_year', 'dist_moscow'],
      dtype='object')

In [17]:
cat_features = ['brand', 'model', 'generation', 'modification', 'equipment', 'body_type', 'color', 'owners_count']
text_features = ['description']

In [14]:
from catboost import CatBoostRegressor

In [21]:
model = CatBoostRegressor(loss_function='RMSE', random_seed=42, eval_metric='MAPE', task_type='GPU', devices='0:1', cat_features=cat_features, text_features=text_features)

In [24]:
grid = {
    'learning_rate': [0.05, 0.1],
    'depth': [6, 8, 10],
    'l2_leaf_reg': [None, 1],
    'iterations': [1000]
}

model.grid_search(
    X=X_train,
    y=y_train,
    param_grid=grid,
    cv=5,
    refit=True,
    partition_random_seed=42,
    search_by_train_test_split=True,
    verbose=True,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.9249516	test: 0.9246088	best: 0.9246088 (0)	total: 279ms	remaining: 4m 39s
1:	learn: 0.8532661	test: 0.8525307	best: 0.8525307 (1)	total: 363ms	remaining: 3m 1s
2:	learn: 0.7982037	test: 0.7969079	best: 0.7969079 (2)	total: 445ms	remaining: 2m 27s
3:	learn: 0.7581098	test: 0.7566735	best: 0.7566735 (3)	total: 525ms	remaining: 2m 10s
4:	learn: 0.7192892	test: 0.7180547	best: 0.7180547 (4)	total: 600ms	remaining: 1m 59s
5:	learn: 0.6926103	test: 0.6912622	best: 0.6912622 (5)	total: 683ms	remaining: 1m 53s
6:	learn: 0.6723585	test: 0.6709333	best: 0.6709333 (6)	total: 764ms	remaining: 1m 48s
7:	learn: 0.6531439	test: 0.6518133	best: 0.6518133 (7)	total: 835ms	remaining: 1m 43s
8:	learn: 0.6289864	test: 0.6277822	best: 0.6277822 (8)	total: 909ms	remaining: 1m 40s
9:	learn: 0.6163904	test: 0.6149362	best: 0.6149362 (9)	total: 987ms	remaining: 1m 37s
10:	learn: 0.6023530	test: 0.6008073	best: 0.6008073 (10)	total: 1.06s	remaining: 1m 35s
11:	learn: 0.5906965	test: 0.5891447	best:

{'params': {'depth': 10,
  'l2_leaf_reg': 1,
  'iterations': 1000,
  'learning_rate': 0.05},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               

In [32]:
pd.Series(model.predict(X_test)).to_csv('solution5.csv', header=False, index=False)

In [33]:
model.save_model('catboost_model', format='cbm')

In [34]:
from catboost import CatBoost
model = CatBoost()
model = model.load_model('catboost_model', format='cbm')